In [2]:
import cv2
import mediapipe as mp
import urllib.request as urlreq
import os
from pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression

In [3]:
def readtps(input):
    """
    Function to read a .TPS file
    Args:
        input (str): path to the .TPS file
    Returns:
        lm (str list): info extracted from 'LM=' field
        im (str list): info extracted from 'IMAGE=' field
        id (str list): info extracted from 'ID=' filed
        coords: returns a 3D numpy array if all the individuals have same
                number of landmarks, otherwise returns a list containing 2d
                matrices of landmarks
    """

    # open the file
    tps_file = open(input, 'r')  # 'r' = read
    tps = tps_file.read().splitlines()  # read as lines and split by new lines
    tps_file.close()

    # initiate lists to take fields of "LM=","IMAGE=", "ID=" and the coords
    lm, im, ID, coords_array = [], [], [], []

    # looping thru the lines
    for i, ln in enumerate(tps):

        # Each individual starts with "LM="
        if ln.startswith("LM"):
            # number of landmarks of this ind
            lm_num = int(ln.split('=')[1])
            # fill the info to the list for all inds
            lm.append(lm_num)
            # initiate a list to take 2d coordinates
            coords_mat = []

            # fill the coords list by reading next lm_num of lines
            for j in range(i + 1, i + 1 + lm_num):
                coords_mat.append(tps[j].split(' '))  # split lines into values

            # change the list into a numpy matrix storing float vals
            coords_mat = np.array(coords_mat, dtype=float)
            # fill the ind 2d matrix into the 3D coords array of all inds
            coords_array.append(coords_mat)
            # coords_array.append(coords_mat)

        # Get info of IMAGE= and ID= fields
        if ln.startswith("IMAGE"):
            im.append(ln.split('=')[1])

        if ln.startswith("ID"):
            ID.append(ln.split('=')[1])

    # check if all inds contains same number of landmarks
    all_lm_same = all(x == lm[0] for x in lm)
    # if all same change the list into a 3d numpy array
    if all_lm_same:
        coords_array = np.dstack(coords_array)

    # return results in dictionary form
    return {'lm': lm, 'im': im, 'id': ID, 'coords': coords_array}

In [4]:
def LBF_model(image, prepared = False):
    if not prepared:
        # save face detection algorithm's name as haarcascade
        haarcascade = "haarcascade_frontalface_alt2.xml"

        # save facial landmark detection model's name as LBFmodel
        LBFmodel = "lbfmodel.yaml"

        if (haarcascade not in os.listdir(os.curdir)):
            haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml"
            urlreq.urlretrieve(haarcascade_url, haarcascade)

        if (LBFmodel not in os.listdir(os.curdir)):
            LBFmodel_url = "https://github.com/kurnianggoro/GSOC2017/raw/master/data/lbfmodel.yaml"
            urlreq.urlretrieve(LBFmodel_url, LBFmodel)

        # create an instance of the Face Detection Cascade Classifier
        detector = cv2.CascadeClassifier(haarcascade)
        # create an instance of the Facial landmark Detector with the model
        landmark_detector = cv2.face.createFacemarkLBF()
        landmark_detector.loadModel(LBFmodel)
    
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Detect faces using the haarcascade classifier on the "grayscale image"
    faces = detector.detectMultiScale(image_gray)

    # Detect landmarks on "image_gray"
    _, landmarks = landmark_detector.fit(image_gray, faces)
    landmarks = landmarks[0][0,:,:]
    landmarks = np.divide(landmarks, (image.shape[1], image.shape[0]))
    return landmarks

In [5]:
def MediaPipe_model(image):
    landmarks = np.empty((478, 2))
    mp_face_mesh = mp.solutions.face_mesh

    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,    # Zpřesnění kolem očí a pusy!!
        min_detection_confidence=0.5) as face_mesh:

        # To improve performance
        image.flags.writeable = False

        # Detect the face landmarks
        results = face_mesh.process(image)

        # To improve performance
        #image.flags.writeable = True

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for i, l_mark in enumerate(face_landmarks.landmark):
                landmarks[i,:] = l_mark.x, l_mark.y
    return landmarks

In [33]:
def display_landmarks(landmarks, img, pixel_scale = False):
    x_coor = landmarks[:,0]
    y_coor = landmarks[:,1]
    if not pixel_scale:
        x_coor = np.multiply(x_coor, img.shape[1])
        y_coor = np.multiply(y_coor, img.shape[0])
        
    fig, ax = plt.subplots(1, figsize = (10,10))
    ax.imshow(image)
    #ax.axis('off')

    for x, y in zip(x_coor, y_coor):
        circ = Circle((x,y), 2, color='white', fill = False)
        ax.add_patch(circ)  

    plt.show()

In [7]:
## pixel analysis - hair edge
def hair_edge(image, middle_landmarks):
    forehead = image[0:min(middle_landmarks[:,1]), middle_landmarks[0,0]:middle_landmarks[1,0],:]

    kmeans = KMeans(n_clusters = 3, n_init = 10).fit(forehead.reshape(-1,3))
    forehead_labels = kmeans.labels_.reshape(forehead.shape[0], forehead.shape[1])
    background_label = kmeans.cluster_centers_.mean(axis = 1).argmax()
    non_background_label = kmeans.cluster_centers_.mean(axis = 1).argmin()
    
    Y_coordinate = np.array([range(forehead.shape[0])] * forehead.shape[1]).transpose()
    forehead_labels = forehead_labels.reshape(-1,1)
    Y_coordinate = Y_coordinate.reshape(-1,1)
    
    face_pixels_idx = (forehead_labels != background_label).nonzero()[0]
    Y = Y_coordinate[face_pixels_idx]
    labels = (forehead_labels[face_pixels_idx] == non_background_label).reshape(-1,)
    log_reg = LogisticRegression()
    log_reg.fit(Y, labels)
    decision_boundary = -log_reg.intercept_/log_reg.coef_
    plt.figure(figsize = (10,10))
    plt.imshow(image)
    
    plt.plot(np.arange(image.shape[1]),np.array([decision_boundary] * image.shape[1]).reshape(-1,))
    plt.show()
    return decision_boundary

In [53]:
#Main Pipeline

def prepare_training_landmarks(both_models = True):
    if both_models:
        x = np.empty((0, 546 * 2))
    else:
        x = np.empty((0, 478 * 2))
    y_true = np.empty((0, 144))
    
    # some tree structure here
    groups = ['Males', 'Females']
    path = 'C:\\Users\\Zirov\\landmarks\\'
    
    for group in groups:
        tps = readtps(path + group + '.TPS')
        
        for idx in range(len(tps['im'])):
            print(f'Group: {group}, index: {idx}', end = '\r')
            true_landmarks = tps['coords'][:, :, idx]
            img_path = path + group + '\\' + tps['im'][idx]
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            true_landmarks = np.divide(true_landmarks, (image.shape[1], image.shape[0]))

            # Both model use float(0,1) for x and y axis
            if both_models:
                input_landmarks = np.concatenate((LBF_model(image), MediaPipe_model(image)), axis = 0)
            else:
                input_landmarks = MediaPipe_model(image)
                
            # batch_dim = 0
            input_landmarks = input_landmarks.reshape(1,-1)
            x = np.concatenate((x, input_landmarks), axis = 0)
            true_landmarks = true_landmarks.reshape(1,-1)
            y_true = np.concatenate((y_true, true_landmarks), axis = 0)
    return x, y_true

In [54]:
x,y = prepare_training_landmarks()

In [55]:
x.shape

(259, 1092)

In [56]:
y.shape

(259, 144)